<a href="https://colab.research.google.com/github/EvgenyEsin/Spark_Apache/blob/main/Spark_Apache_sem_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ДЗ к семинару 2
Условие: дана таблица с колонками (id, name, salary, managerid). Необходимо написать код на spark, который создает эту таблицу (данные указаны ниже) и в результате выдаст таблицу, в которой будут имена сотрудников, которые зарабатывают больше своих менеджеров



In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488490 sha256=0f2add60a30dbd2f2de5dda033321519d762cc0ed9fe6f77845a08c6d989540c
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pandas as pd

Исходные данные:

In [3]:
data = {'id': [1,2,3,4],
        'Name': ['Joe', 'Henry', 'Sam', 'Max'],
        'Salary': ['70', '80', '60', '90'],
        'ManagerId': ['3', '4', 'Null', 'Null']}
df_pd = pd.DataFrame(data)
df_pd

,id,Name,Salary,ManagerId
0,1,Joe,70,3
1,2,Henry,80,4
2,3,Sam,60,Null
3,4,Max,90,Null


Создаем DataFrame:

In [4]:
spark = SparkSession.builder.appName('Homework2').getOrCreate()

df = spark.createDataFrame(df_pd)
df.show()

+---+-----+------+---------+
| id| Name|Salary|ManagerId|
+---+-----+------+---------+
|  1|  Joe|    70|        3|
|  2|Henry|    80|        4|
|  3|  Sam|    60|     Null|
|  4|  Max|    90|     Null|
+---+-----+------+---------+



Создаем вспомогательный датафрейм

In [5]:
managers_df = df.select('id', 'Salary').withColumnRenamed('id', 'manager_id').withColumnRenamed('Salary', 'manager_salary')
managers_df.show()

+----------+--------------+
|manager_id|manager_salary|
+----------+--------------+
|         1|            70|
|         2|            80|
|         3|            60|
|         4|            90|
+----------+--------------+



In [6]:
joined = df.join(managers_df, df['ManagerId'] == managers_df['manager_id'], 'inner')
joined.show()

+---+-----+------+---------+----------+--------------+
| id| Name|Salary|ManagerId|manager_id|manager_salary|
+---+-----+------+---------+----------+--------------+
|  1|  Joe|    70|        3|         3|            60|
|  2|Henry|    80|        4|         4|            90|
+---+-----+------+---------+----------+--------------+



Фильтруем по условию и выводим результат:

In [7]:
joined.filter(joined['Salary'] > joined['manager_salary']).select(joined.Name).show()

+----+
|Name|
+----+
| Joe|
+----+

